In [66]:
import numpy as np  
import pandas as pd 
import os


data = []

for dirpath, dirnames, filenames in os.walk('data\\'):
    for filename in filenames:
        data.append(os.path.join('data', filename))

data


['data\\gender_submission.csv', 'data\\test.csv', 'data\\train.csv']

### Data check and organization

In [67]:
gender_submission_path, test_path, train_path = data


train = pd.read_csv(train_path)

test = pd.read_csv(test_path)


train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


### Finding and filling nan values

In [68]:
train['Age'].isna().sum()
train['Cabin'].isna().sum()
train['Embarked'].isna().sum()

#print(train['Age'].value_counts(dropna=False))
train['Age'].describe()
#Age mean is 29.699118 so i'll replace all nan's in Age column with this value,
#don't know if this is the right move, buti know it's better than zero

train['Cabin'].value_counts(dropna=False)
#CHANGE OF PLANS, I DROP ALL THE COLUMNS WHERE CABIN IS NAN 
#train = train.dropna(subset=['Cabin'])
#still don't know what to do with this one theres a lot of nans 687 out of 891
#as of right now i'll just ignore this column even tho it seems important.
 
train['Embarked'].value_counts(dropna=False)
#theres only 2 nans so im probably gonna drop the records, it's nt a big loss
train = train.dropna(subset=['Embarked'])



train = train.fillna({'Age':29.699118})

### X and Y extraction

In [69]:
y = train['Survived']

# all columns
x_columns = ['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']

X_pre_encoding = train[x_columns]
X_pre_encoding.isna().sum()
X_pre_encoding.count()

PassengerId    889
Pclass         889
Name           889
Sex            889
Age            889
SibSp          889
Parch          889
Ticket         889
Fare           889
Cabin          202
Embarked       889
dtype: int64

### name and ticket label encoding, one hot encoding

In [70]:
from sklearn.preprocessing import LabelEncoder

def encode(df):
    encoder = LabelEncoder()
    #df.loc[:,'names_encoded'] = encoder.fit_transform(df['Name'])

    #df.loc[:,'ticket_encoded'] = encoder.fit_transform(df['Ticket'])

    df = df.drop(columns=['Name', 'Ticket', 'Cabin'])

    df = pd.get_dummies(df, columns=['Sex', 'Embarked'])

    return df


X = encode(X_pre_encoding)

X

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,1,3,22.000000,1,0,7.2500,False,True,False,False,True
1,2,1,38.000000,1,0,71.2833,True,False,True,False,False
2,3,3,26.000000,0,0,7.9250,True,False,False,False,True
3,4,1,35.000000,1,0,53.1000,True,False,False,False,True
4,5,3,35.000000,0,0,8.0500,False,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...
886,887,2,27.000000,0,0,13.0000,False,True,False,False,True
887,888,1,19.000000,0,0,30.0000,True,False,False,False,True
888,889,3,29.699118,1,2,23.4500,True,False,False,False,True
889,890,1,26.000000,0,0,30.0000,False,True,True,False,False


### exchanging cabin names between train data and test data, encoding test data

In [71]:
test = encode(test)

In [72]:
test_cabins = [col for col in test.columns if col.startswith('Cabin')] 
train_cabins = [col for col in X.columns if col.startswith('Cabin')] 


print(len(test_cabins))
print(len(train_cabins))


for cab in test_cabins:
    if cab not in train_cabins:
        X[cab] = False
        train_cabins.append(cab)
        
for cab in train_cabins:
    if cab not in test_cabins:
        test[cab] = False


0
0


### sorting columns and last check

In [73]:
'''test = test.sort_index(axis=1)
X = X.sort_index(axis=1)

test.to_csv('test')
X.to_csv('X')

print(len(test.columns))
print(len(X.columns))'''

11
11


### Creating model and fitting data

In [74]:
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier

model = DecisionTreeRegressor(random_state = 1)

model.fit(X, y)

DecisionTreeRegressor(random_state=1)

### testing the model - first prediction

In [75]:
results = model.predict(test)

results_int = []
for value in results:
    results_int.append(int(value))

print(len(results_int))
print(results_int)




418
[0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1,

### exporting results into a CSV file

In [76]:
def export_results(prediciton_df):
    
    data = []
    temp_data = []

    for dirpath, dirnames, filenames in os.walk('submissions\\'):
        for filename in filenames:
            data.append(os.path.join('submissions', filename))
            temp_data.append(filename.removesuffix('.csv'))

    name_split = temp_data[-1].split('_')
    #print(data)
    
    
    new_sub_name = ''.join(['submissions\\submission_', str(int(name_split[-1]) + 1), '.csv'])
    data.append(new_sub_name)
    
    #print(new_sub_name)
    prediciton_df['Survived'] = results_int
    prediciton_df.to_csv(path_or_buf=new_sub_name, columns=['PassengerId', 'Survived'], index=False)
    
    return data
    
submissions_list = export_results(test)

### additional functions to compare submisions

In [77]:
def comapre_submissions(sub1, sub2):
    sub_a = pd.read_csv(sub1)
    sub_b = pd.read_csv(sub2)
    
    different_predictions = 0
    
    for a, b in zip(sub_a['Survived'], sub_b['Survived']):
        if a != b: different_predictions += 1
        
    return different_predictions
    
    
comapre_submissions(submissions_list[1], submissions_list[5])

64